# Psycopg2

In [6]:
%pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect("dbname=dvdrental user=postgres password=postgres")

In [5]:
from pprint import pprint

cur = conn.cursor()
cur.execute("SELECT film_id FROM film;")
# pprint(cur.fetchall())

In [8]:
pprint(cur.fetchone())

(8,)


## Placeholder

In [22]:
cur.execute(
    "select * from film \
    where title ilike %s and length > %s",
   ("%ski%", 10)
)
pprint(cur.fetchall())

[(514,
  'Lebowski Soldiers',
  'A Beautiful Epistle of a Secret Agent And a Pioneer who must Chase a '
  'Astronaut in Ancient China',
  2006,
  1,
  6,
  Decimal('2.99'),
  69,
  Decimal('17.99'),
  'PG-13',
  datetime.datetime(2013, 5, 26, 14, 50, 58, 951000),
  ['Commentaries', 'Deleted Scenes'],
  "'agent':9 'ancient':19 'astronaut':17 'beauti':4 'chase':15 'china':20 "
  "'epistl':5 'lebowski':1 'must':14 'pioneer':12 'secret':8 'soldier':2")]


In [23]:
cur.execute(
    "select * from film \
    where title ilike %(pattern)s and length > %(min_length)s",
   {"pattern": "%ski%", "min_length": 10}
)
pprint(cur.fetchall())

[(514,
  'Lebowski Soldiers',
  'A Beautiful Epistle of a Secret Agent And a Pioneer who must Chase a '
  'Astronaut in Ancient China',
  2006,
  1,
  6,
  Decimal('2.99'),
  69,
  Decimal('17.99'),
  'PG-13',
  datetime.datetime(2013, 5, 26, 14, 50, 58, 951000),
  ['Commentaries', 'Deleted Scenes'],
  "'agent':9 'ancient':19 'astronaut':17 'beauti':4 'chase':15 'china':20 "
  "'epistl':5 'lebowski':1 'must':14 'pioneer':12 'secret':8 'soldier':2")]


## Transaction

In [25]:
cur.execute(
    "update my_film set title = 'بادیگارد' where title ilike '%ski%'"
)


In [26]:
conn.commit()

In [27]:
cur.execute(
    "select * from my_film where title = %s", ("بادیگارد",)
)
pprint(cur.fetchall())

[(514,
  'بادیگارد',
  'A Beautiful Epistle of a Secret Agent And a Pioneer who must Chase a '
  'Astronaut in Ancient China',
  2006,
  1,
  6,
  Decimal('2.99'),
  69,
  Decimal('17.99'),
  'PG-13',
  datetime.datetime(2013, 5, 26, 14, 50, 58, 951000),
  ['Commentaries', 'Deleted Scenes'],
  "'agent':9 'ancient':19 'astronaut':17 'beauti':4 'chase':15 'china':20 "
  "'epistl':5 'lebowski':1 'must':14 'pioneer':12 'secret':8 'soldier':2")]


In [30]:
cur.execute(
    "update my_film set title = 'به وقت شام' where title ilike '%بادیگارد%'"
)

In [31]:
conn.commit()
conn.rollback() # nonsense

In [33]:
try:
    cur.execute(
        "some bad sql"
    )
    # ...
except Exception as e:
    conn.rollback()
    print(e)
    print("rolled back")
else:
    conn.commit()
    print("commited")


cur.execute(
        "another sql ..."
    )
cur.commit()

syntax error at or near "some"
LINE 1: some bad sql
        ^

rolled back


In [35]:
conn.close()

Use "with" in order to commit / roll back automatically.

In [37]:
username = "postgres"
password = "postgres"
connect_config = f"dbname=dvdrental user={username} password={password}"
conn = psycopg2.connect(connect_config)

try:
    with conn:
        with conn.cursor() as curs:
            curs.execute('SELECT * FROM my_film;')
finally:
    conn.close() # Yet needs closing